In [1]:
import onnx
import copy
import numpy as np
import onnxruntime as ort

In [2]:
model = onnx.load("./good_nn.onnx")
in_name = "onnx::Gemm_0"

In [3]:
weight_matrix = copy.deepcopy(model.graph.initializer[0])

In [4]:
weight_matrix

dims: 64
dims: 2
data_type: 1
name: "policy_net.0.weight"
raw_data: "\2258\323>\202k$>\301\035\377\275)8\306=\226\233:\276$\\\314\275\376]\327=\000\351\225\276Pnm>\221\363\210>Y\337\312\275$\254\331>\024\262\036;\331\374\275>\024\276\374<\353|s>?j0>;\010\221\276\236V\200\276\310\000<>\321\220\272\274y\273\036<\207\247\223\275\256\035g\276\275R0\275\327\306\230>\024\276A\2766\013\205\276\340+\252\275\177\222\002\276\323\356\003\276\004W\307\274\217\332\261\2769\315\216>\000\316O\276\020\003~\276W)\002>\003l\210\276\332\371!\276\317w\362;\302D\205=Q\260\375\276\036\217\210\276\3127\220\276\252\001\024>\274h\234\276!\001_>\272\303:>\270\313\025=\317\245\200\276\207\376V=\303t\212=\261\031\024>h\270o>+`\037>^U\242>\357D:\276\204D\177>K\'\233:^\200\370\274`]\332=\220\254\022\2768\313\255=%\345$\276C\356\202\276C\365\335=b\240\205\275WaY=@\004\272\276e\265n\274\037\322\301\275\341Qw>\000\021$>\331C\005\277\3009W=\'\240R\276\376\235\230\275\232\363\352\275\305\222\232=\327\337\247\275\216\016

In [5]:
arr=np.zeros((2+3,2))
arr[0,0]=1
arr[1,1]=1
initializer_tensor = onnx.helper.make_tensor(
name="WInit",
data_type=onnx.TensorProto.FLOAT,
dims=arr.shape,
vals=arr.flatten().tolist())

In [6]:
init_layer = onnx.helper.make_node(
    "MatMul",
    inputs=[in_name, "WInit"],
    outputs=["XInit"]
)

In [7]:
model.graph.node.insert(0,init_layer)
model.graph.node[1].input[0] = "XInit"
model.graph.initializer.insert(0,initializer_tensor)
#model.graph.input[0].type.tensor_type.shape.dim[0].dim_value=2+3
model.graph.input[0].type.tensor_type.shape.dim[1].dim_value=2+3

In [8]:
onnx.save(model, "./good_nn-num.onnx")

In [11]:
ort_sess_orig = ort.InferenceSession("./good_nn.onnx")
ort_sess_num = ort.InferenceSession("./good_nn-num.onnx")
x = np.random.uniform(size=2)
x = np.array(x,dtype=np.float32)
eps = np.zeros(3)
print(x)
print(eps)
x_num = np.concatenate([x,eps],dtype=np.float32)
print(x_num)
print(ort_sess_orig.run(None, {"onnx::Gemm_0": [x]}))
print(ort_sess_num.run(None, {"onnx::Gemm_0": [x_num]}))

[0.3631676  0.23652671]
[0. 0. 0.]
[0.3631676  0.23652671 0.         0.         0.        ]
[array([[-1.116547  , -0.22697537,  0.6658399 ]], dtype=float32)]
[array([[-1.116547  , -0.22697537,  0.6658399 ]], dtype=float32)]
